### Ejercicio 3
Una de las aplicaciones más interesantes que se pueden realizar sobre la información que se recopila de Twitter es el denominado análisis de sentimiento o sentiment analysis. Se trata fundamentalmente de clasificar de forma automática la polaridad de un tweet en concreto, en fucnión de su contenido, categorizándolo como positivo, negativo o neutro, e incluso asignándole un valor numérico a dicha clasificación. Esta funcionalidad se utiliza, por ejemplo, en sistemas de análisis de reputación online (Online Reputation Managment, ORM) para determinar qué se está diciendo de una determinada marca en Twitter, en tiempo real.

En este ejercicio vamos a simular, de forma simplificada, un sistema de tiempo real que ofrezca una puntuación de polaridad (positiva, negativa o neutra) a los tweets que vayan llegando. Para ello, el modelo de clasificación se basará en listas de palabras, denominadas lexicones. En concreto, dispondremos de un lexicón d palabras positivas y su puntuación, "positive_lex.txt", y de un lexicón de palabras negativas y su puntuación, "negative_lex.txt". Ambos contienen, en cada línea, una palabra (positiva o negativa, respectivamente), seguida de su puntuación. Nótese que la polaridad negativa ya está expresada con un signo menos.

Se pide desarrollar un notebook de Jupyter, denominado "sentiment.ipynb", en el que se utilice como fuente de datos Kafka, y en concreto el topic kafkaTwitter.

Para cada nuevo tweet que llegue al sistema, se analizará su polaridad dividiendo el tweet por palabras y realizando la suma de la polaridad de todas las laplabras que aparezcan en los ficheros "positive_lex.txt" y "negative_lex.txt". Las palabras de lexicon negativo tienen un signo menos delante, por lo que restará a la puntuación final del tweet. Si ninguna de las palabras del tweet está almacenada en los ficheros proporcionados (o si las puntuaciones de las palabras positivas y negativas se anulan), la polaridad del tweet será 0.

La salida del sistema ha de imprimirse cada segundo, y consistirá en una línea por cada tweet recibido, que contenga el texto del propio tweet, a continuación la puntuación de su polaridad, y la palabra "NEUTRO" si la puntuación es igual a 0, "POSITIVO" si es mayor que 0, y "NEGATIVO" si es menor que 0.

Por ejemplo, tenemos el siguiente tweet:

"Hoy vuelvo a España, me ha encantado Polonia :D'

El sistema devolverá el texto del tweet, seguido de la puntuación 0.417, y de la palabra "POSITIVO", ya que la palabra "encantado" está en el fichero "positive_lex.txt" con esa puntuación, y el resto de las palabras no se encuentran en ninguno de los dos lexicones.

Se recomienda modificar el ritmo de volcado de tweets a Kafka, para que la salida sea más legible. Para ello, modificar la línea 24 del script "tweet_producer.py" de sleep(random.random()/10) a sleep(1).

In [17]:
import findspark
import requests
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
from pyspark.streaming.kafka import KafkaUtils
import os
from collections import namedtuple
import pandas as pd
import time
from IPython import display

pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [39]:
# ubicación de spark
findspark.init('/home/j/spark-2.4.7-bin-hadoop2.7')

In [40]:
# necesario para cargar la librería de spark streaming y kafka
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.7 pyspark-shell'

In [42]:
# nombre del topic
topic = 'kafkaTwitter'

In [43]:
# creación de los contextos de spark
sc = SparkContext("local[*]")
ssc = StreamingContext(sc, 5)
sqlContext = SQLContext(sc)

In [44]:
# creación del DStream a partir de Kafka
kafkaStream = KafkaUtils.createDirectStream(ssc, [topic], {
                        'bootstrap.servers':'localhost:9092, localhost:9093',
                        'group.id':'exercise3'})

In [45]:
# ventana de recepción de tweets de 20 segundos
lines = kafkaStream.window( 20 )

In [46]:
# estrucutra de la tupla de los tweets
fields = ("content")
Tweet = namedtuple( 'Tweet', fields )

In [47]:
# Obtención de los tweets
( lines.map( lambda rec: Tweet( rec[1] ) ) # Almacenar los tweets en un objeto
  .foreachRDD( lambda rdd: rdd.toDF() # transformarlos en dataframe
  .limit(1).registerTempTable("tweets") ) # registrarlos en una tabla
) 

In [48]:
# estructura de la tupla de los lexicones
fields = ("word","score")
Lexicon = namedtuple( 'Lexicon', fields )

In [50]:
# obtención del lexicón positivo y su almacenado en una tabla
positive_lex_rdd = sc.textFile("/home/j/spark-2.4.7-bin-hadoop2.7/python/TP3/resources/positive_lex.txt")
positive_lex_rdd_pl = ( positive_lex_rdd
  .map ( lambda rec: Lexicon(rec.split(" ")[0], rec.split(" ")[1]))
)
positive_lex_df = sqlContext.createDataFrame(positive_lex_rdd_pl)
positive_lex_df.registerTempTable('positive_lex')

In [51]:
# obtención del lexicón negativo y su almacenado en una tabla
negative_lex_rdd = sc.textFile("/home/j/spark-2.4.7-bin-hadoop2.7/python/TP3/resources/negative_lex.txt")
negative_lex_rdd_pl = ( negative_lex_rdd
  .map ( lambda rec: Lexicon(rec.split(" ")[0], rec.split(" ")[1]))
)
negative_lex_df = sqlContext.createDataFrame(negative_lex_rdd_pl)
negative_lex_df.registerTempTable('negative_lex')

In [52]:
# inicio del stream
ssc.start()

In [ ]:
# mientras sea verdad continuar con el stream
while True:
    # le pongo un temporizador de 2 segundos por tweet
    time.sleep( 2 )
    
    # cargo el tweet
    tweet = sqlContext.sql( '''
        select
            content
        from tweets
    ''' )
    
    # creo un dataframe del tweet
    tweet_df = tweet.toPandas()
    
    # imprimo el tweet
    print("Tweet: ",tweet_df['content'].to_string(index=False))
    
    # creo lista para almacenar las puntuaciones del tweet
    total_score = []
    
    # recorro las palabras del tweet
    for word_list in tweet_df['content'].str.split():
        for word in word_list:
            
            # para cada palabra busco si se encuentra en los lexicones: 
            # - esto lo podía haber hecho de múltiples maneras, incluso más eficientes usando diccionarios etc, pero 
            #   quería utilizar spark sql para recorrer los lexicones
            word_score1 = sqlContext.sql( '''
                select
                    sum(score) as word_score
                from
                    (
                    select
                        score
                    from positive_lex
                    where word = "'''+word.replace('"','').lower()+'''"
                    union
                    select
                        score
                    from negative_lex
                    where word = "'''+word.replace('"','').lower()+'''"
                    )
            ''' )
            
            # almacenamos las puntuaciones en la lista 
            total_score.append(word_score1.first())

        # creamos un dataframe con las puntuaciones
        total_score_df = pd.DataFrame(total_score,columns=['word_score'])
        
        # hacemos la suma de las puntuaciones
        tweet_score = total_score_df['word_score'].sum()
        
        # imprimios la puntuación
        print("Tweet score: ", tweet_score)
        
        # por último imprimos el sentimiento en función de la puntuación
        if(tweet_score==0):
            print("Tweet sentiment: NEUTRAL")
        if(tweet_score<0):
            print("Tweet sentiment: NEGATIVE")
        if(tweet_score>0):
            print("Tweet sentiment: POSITIVE")

    # borramos el log de los tweets procesados anteriormente
    display.clear_output(wait=True)
    
    # nota: dejo la salida de un tweet y su resultado

Tweet:   'The Big Bang Theory' crea un programa de becas para ayudar a universitarios científicos http://t.co/TnJzDoSbEo http://t.co/9aC103yQjt
Tweet score:  -0.09999999999999998
Tweet sentiment: NEGATIVE


In [38]:
sc.stop()
ssc.stop()